# How to Sign and Send Transactions on Terra

In [ ]:
from terra_sdk.core.bank import MultiSendInput, MultiSendOutput, MsgMultiSend, MsgSend
from terra_sdk.client.lcd.api.tx import CreateTxOptions
from terra_sdk.client.localterra import LocalTerra
from terra_sdk.key.mnemonic import MnemonicKey
from terra_sdk.client.lcd import LCDClient
from terra_sdk.core import Coins, Coin
import requests, json

### Initialize Client & Wallet

**Note:** _Only uncomment the code block which corresponds to the network on which you would like to carry out transactions and comment out the other code blocks. In this notebook, LocalTerra is used by default and is the recommended network to use for testing purposes._

##### LocalTerra

In [ ]:
# Create client to communicate with localterra.
terra = LocalTerra()

# Initialize preconfigured test wallet.
wallet = terra.wallets["test1"]

##### Testnet

In [ ]:
# # Create client to communicate with testnet.
# terra = LCDClient(
#     url="https://bombay-lcd.terra.dev/",
#     chain_id="bombay-12"
# )

# # Initialize wallet with associated mnemonic key.
# mk = MnemonicKey(mnemonic="<INSERT MNEMONIC KEY HERE>")
# wallet = terra.wallet(mk)

##### Mainnet

In [ ]:
# # Create client to communicate with mainnet.
# terra = LCDClient(
#     url="https://lcd.terra.dev",
#     chain_id="columbus-5"
# )

# # Initialize wallet with associated mnemonic key.
# mk = MnemonicKey(mnemonic="<INSERT MNEMONIC KEY HERE>")
# wallet = terra.wallet(mk)

### Set & View Gas Prices

In [ ]:
# Request current gas rates for future fee estimation.
gas_price_dict = requests.get("https://pisco-api.terra.dev/gas-prices").json()
gas_price_dict

---

## One-to-One Wallet Transactions

---

### Set Relevant Transaction Parameters

In [ ]:
# Initialize sender and recipient wallet addresses.
sender_address = wallet.key.acc_address
recipient_address = terra.wallets["test2"].key.acc_address

# Set relevant parameters for transaction.
tx_options = CreateTxOptions(
    msgs=[
        MsgSend(
            from_address=sender_address,
            to_address=recipient_address,
            amount=Coins([Coin("uluna", 1000000)])
        )
    ],
    gas="auto",
    gas_prices=Coins(gas_price_dict),
    fee_denoms="uluna",
    gas_adjustment=1.5
)

### Create, Sign & Execute Transaction

In [ ]:
# Create and sign transaction.
tx = wallet.create_and_sign_tx(options=tx_options)

# Broadcast the request for execution to the Terra node.
result = terra.tx.broadcast(tx)

### Print Transaction Information

In [ ]:
# Utilize print helper function to neatly print out transaction information.
%run helper-functions.ipynb
print_message(result)

---

## Multisend Transactions

---

### Set Relevant Transaction Parameters

In [ ]:
# Initialize sender and recipient wallet addresses.
sender_address = wallet.key.acc_address
recipient_address = terra.wallets["test2"].key.acc_address
recipient_address2 = terra.wallets["test3"].key.acc_address

# Set input and output variables for multi-send message.
multi_send_inputs = [
    MultiSendInput(sender_address, Coins([Coin('uluna', 1000000)])),
]
multi_send_outputs = [
    MultiSendOutput(recipient_address, Coins([Coin('uluna', 500000)])),
    MultiSendOutput(recipient_address2, Coins([Coin('uluna', 500000)]))
]

# Set relevant parameters for transaction.
tx_options = CreateTxOptions(
    msgs=[
        MsgMultiSend(
            inputs=multi_send_inputs,
            outputs=multi_send_outputs,
        )
    ],
    gas='auto',
    gas_prices=Coins(gas_price_dict),
    fee_denoms='uluna',
    gas_adjustment=1.5
)

### Create, Sign & Execute Transaction

In [ ]:
# Create and sign transaction.
tx = wallet.create_and_sign_tx(options=tx_options)

# Broadcast the request for execution to the Terra node.
result = terra.tx.broadcast(tx)

### Print Transaction Information

**Note:** _Helper functions are only available if helper-functions.ipynb are in the same directory as this Jupyter Notebook. Run from cloned repository to ensure proper function._

In [ ]:
# Utilize print helper function to neatly print out transaction information.
%run helper-functions.ipynb
print_message(result)